# TopAGNPS run sequence

- This notebook executes the commands to:
    - Run the preprocessing step of TopAGNPS from presribed boundary
    - Identify the outlet
    - Finish the computation

In [1]:
# import os
# os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd

from pyagnps import topagnps


## Define paths and select geometry for DEM selection

In [2]:

root_dir = 'C:/Users/Luc/projects/pyagnps/outputs/topagnps/'

run_dir_name = 'thuc_1139'

path_to_shapefile = 'C:/Users/Luc/projects/pyagnps/inputs/topagnps/johnson_creek_long_creek/johnshon_long_creek_boundary.zip'

path_to_thucs = 'C:/Users/Luc/projects/pyagnps/inputs/thucs/tophuc_S_M_40000_closed_holes_with_container_thuc_merged_bbox_area_first_kept.gpkg'

In [3]:
geom = gpd.read_file(path_to_shapefile)
geom = geom[geom['AREA'] == geom['AREA'].max()].reset_index()

In [4]:
thucs = gpd.read_file(path_to_thucs)
thuc_id = '1139'

geom = thucs[thucs['tophucid']==thuc_id]

## Download/Read DEM and write to `.asc` file

In [8]:
import importlib
importlib.reload(topagnps);

path_to_dir = topagnps.create_topagnps_directory(root_dir, run_dir_name)

path_to_asc = f'{path_to_dir}/thuc_{thuc_id}_res_30_m.asc'
# dem, path_to_asc = topagnps.download_dem(geom, path_to_dir, name=run_dir_name, resolution_m=30, buffer_m=500)
# dem, path_to_asc = topagnps.get_dem_from_raster(geom, path_to_input_raster, path_to_dir, name=thucid_dir_name, resolution_m=10, buffer_m=500, keeptif=True)

## Define `TOPAGNPS.XML` control file

Define the parameters for the preprocessing step

In [9]:
dem_filename = path_to_asc.rsplit('/',1)[-1] # Part of the string after the last / = "thuc_1173_rest_10_m.asc"

topagnpsXML = {'DEMPROC': 2,
               'FORMAT': 0,
               'CSA': 10,
               'MSCL': 250,
               'KEEPFILES': 1,
               'FILENAME': dem_filename}

topagnps.create_topagnps_xml_control_file(topagnpsXML, path_to_dir+'/TOPAGNPS.XML')

## Run TopAGNPS (pre-processing step)

In [10]:
# path_to_TOPAGNPS_bin = '../../../src/bins/TopAGNPS_v6.00.b.017_release_64-bit_Windows.exe' # absolute or with respect to a sub directory in path_to_dir
path_to_TOPAGNPS_bin = 'C:/Users/Luc/projects/pyagnps/src/bins/TopAGNPS_v6.00.b.026_release_64-bit_Windows.exe'

topagnps.run_topagnps(path_to_dir, path_to_TOPAGNPS_bin, memtrack=False)

## Find outlet using `UPAREA.ASC` file and the t-huc geometry

In [11]:
xout, yout, rowout, colout, raster_crs = topagnps.find_outlet_uparea_shape_intersection(path_to_dir+'/UPAREA.ASC', geom)
print(xout, yout, rowout, colout)
print(raster_crs)

309680.81265638664 4095283.453645436 4253 595
EPSG:32616


## Finish computation

In [ ]:
# topagnpsXML = {'DEMPROC': 0,
#                'FORMAT': 0,
#                'CSA': 10,
#                'MSCL': 250,
#                'KEEPFILES': 1,
#                'OUTROW': rowout,
#                'OUTCOL': colout,
#                'READOUT': 1,
#                'FILENAME': dem_filename}

topagnpsXML = {'DEMPROC': 0,
               'FORMAT': 0,
               'CSA': 10,
               'MSCL': 250,
               'KEEPFILES': 1,
               'OUTROW': rowout,
               'OUTCOL': colout,
               'READOUT': 1,
               'FILENAME': 'RELIEF_INP.ASC'}

topagnps.create_topagnps_xml_control_file(topagnpsXML, path_to_dir+'/TOPAGNPS.XML')
topagnps.run_topagnps(path_to_dir, path_to_TOPAGNPS_bin)